In [2]:
import pandas as pd

In [5]:
# Input the dataset
df1 = pd.read_csv('demo/ethereum.csv')
df1.head(5)


FileNotFoundError: [Errno 2] No such file or directory: 'demo/ethereum.csv'

In [ ]:
#Check number of rows and columns, type of each columns
df1.info()

In [ ]:
# Understanding the numeric fields
df1.describe()



In [ ]:

# Histogram of all numeric fields
df_hist = df1.drop(columns=['Date'],axis=1)
df_hist.hist(figsize=(15,15));

In [ ]:

# Visualise the target variable
plt.plot(df1['Date'], df1['Private Domestic (Price Index)'])


In [ ]:

df_series = df.set_index('Date').asfreq('D')
series = pd.Series(df_series['Private Domestic (Price Index)'], index= df_series.index)
results = seasonal_decompose(series, model='additive',freq = 365)
pyplot.rcParams['figure.figsize'] = (20.0, 10.0)
results.plot()
pyplot.show()

In [ ]:


# ACF Plot of Target Variable
plot_acf(series,lags= 100)
pyplot.show()


In [ ]:


# PACF plot of target variable
plot_pacf(series, lags=50)
pyplot.show()


In [ ]:


# Check if any dates are missing
daily_data = pd.DataFrame(pd.date_range(start=df1['Date'].min(),end=df1['Date'].max()))
daily_data.rename(columns={ daily_data.columns[0]: "Date" }, inplace = True)
daily_data.describe()


In [ ]:


# Add the missing dates
df2 = pd.merge(df1,daily_data,on='Date',how='outer')
df2 = df2.sort_values(by=['Date'])
df2.head(10)


In [ ]:
#Use linear interpolation to fill up nulls
df3 = df2.interpolate(method='linear', axis=0).ffill().bfill()
df3.head(10)


In [ ]:
# Outlier Detection using Inter Quartile Range
def out_iqr(s, k=1.5, return_thresholds=False):
    """
    Return a boolean mask of outliers for a series
    using interquartile range, works column-wise.
    param k:
        some cutoff to multiply by the iqr
    :type k: ``float``
    param return_thresholds:
        True returns the lower and upper bounds, good for plotting.
        False returns the masked array 
    :type return_thresholds: ``bool``
    """
    # calculate interquartile range
    q25, q75 = np.percentile(s, 25), np.percentile(s, 75)
    iqr = q75 - q25
    # calculate the outlier cutoff
    cut_off = iqr * k
    lower, upper = q25 - cut_off, q75 + cut_off
    if return_thresholds:
        return lower, upper
    else: # identify outliers
        return [True if x < lower or x > upper else False for x in s]
    
    
# For comparison, make one array each at varying values of k.
df4 = df3.drop(columns=['Date'],axis=1)
iqr1 = df4.apply(out_iqr, k=1.5)
iqr1.head(10)

In [ ]:
for column in df4:
    df4[column] = np.where(iqr1[column] == True,'NaN',df4[column])
cols = df4.columns
df4[cols] = df4[cols].apply(pd.to_numeric, errors='coerce')
df4.head(10)

In [ ]:
#Use linear interpolation to fill up nulls
df = df4.interpolate(method='linear', axis=0).bfill().ffill()
df3['Date'] = pd.to_datetime(df3['Date'])
df = pd.concat([df3['Date'],df], axis=1)
df.head(10)